In [4]:
#Laster inn oversikt over event og event id

import pandas as pd

linker = pd.read_csv("nc_link.csv", delimiter= ';')

In [5]:
#modifiserer linker
import re

#Små bokstaver, fjerner tegn
def modify_event(event):
    # Erstatt mellomrom med underscore (_)
    modified_event = event.replace(' ', '_')
    # Gjør alt til små bokstaver
    modified_event = modified_event.lower()
    # Fjern spesialtegn som / - ( og punktum
    modified_event = re.sub(r'[^a-z0-9_]', '', modified_event)
    return modified_event

#fjerner / i sesong
def modify_sesong(sesong):
    modified_sesong = re.sub(r'[^a-z0-9_]', '', sesong)
    return modified_sesong

#Gjør endringene
linker['event'] = linker['event'].apply(modify_event)
linker['sesong'] = linker['sesong'].apply(modify_sesong)

#Lager en csv nacn
linker['csv_navn'] = linker['sesong']+linker['event']


In [26]:
#api spørringer
resultatliste_url = 'https://live.eqtiming.com/api//Report/220'
resultatliste_per_klasse_url = 'https://live.eqtiming.com/api//Report/346'
deltakerliste_url = 'https://live.eqtiming.com/api//Report/297'

In [23]:
import os
import io
import requests


def get_dataframe_from_api(url, event_id):
    params = {'eventId': event_id}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        # Les dataene direkte inn i en DataFrame
        df = pd.read_csv(io.StringIO(response.content.decode('utf-8')),delimiter=';')
        return df
    else:
        print(f"Feil ved nedlasting av data fra {url}")
        return None
    

In [38]:
def nc_data(event_id):

    resultatliste_url = 'https://live.eqtiming.com/api//Report/220'
    deltakerliste_url = 'https://live.eqtiming.com/api//Report/297'

    params = {'eventId': event_id}

    response_resultatliste = requests.get(resultatliste_url, params=params)
    response_deltakerliste = requests.get(deltakerliste_url, params=params)

    if response_resultatliste.status_code == 200:
        # Les dataene direkte inn i en DataFrame
        resultatliste_df = pd.read_csv(io.StringIO(response_resultatliste.content.decode('utf-8')),delimiter=';')
    else:
        print(f"Feil ved nedlasting av data fra {resultatliste_url}")
        return None
    
    if response_deltakerliste.status_code == 200:
        # Les dataene direkte inn i en DataFrame
        deltakerliste_df = pd.read_csv(io.StringIO(response_deltakerliste.content.decode('utf-8')),delimiter=';')
    else:
        print(f"Feil ved nedlasting av data fra {deltakerliste_url}")
        return None
    
    filtered_df = deltakerliste_df[deltakerliste_df['REGION'] == 'Troms SSK'].copy() 
    filtered_df.loc[:, 'navn'] = filtered_df['FIRSTNAME'].str.cat(filtered_df['SURNAME'], sep=' ')
    navn_list = filtered_df['navn'].tolist()
    resultatliste_df['navn'] = resultatliste_df['Firstname'].str.cat(resultatliste_df['Surname'], sep=' ')
    filtered_resultatliste_df = resultatliste_df[resultatliste_df['navn'].isin(navn_list)].copy() 

    return filtered_resultatliste_df

In [39]:
test = nc_data(38552)
test_2 = nc_data(21041)

In [40]:
resultatliste_df = get_dataframe_from_api(resultatliste_per_klasse_url, 38552)
deltakerliste_df = get_dataframe_from_api(deltakerliste_url, 38552)
test = get_dataframe_from_api(resultatliste_url, 38552)

In [21]:
filtered_df = deltakerliste_df[deltakerliste_df['REGION'] == 'Troms SSK']
filtered_df['navn'] = filtered_df['FIRSTNAME'].str.cat(filtered_df['SURNAME'], sep=' ')
navn_list = filtered_df['navn'].tolist()
resultatliste_df['navn'] = resultatliste_df['Firstname'].str.cat(resultatliste_df['Surname'], sep=' ')
filtered_resultatliste_df = resultatliste_df[resultatliste_df['navn'].isin(navn_list)]



/var/folders/rr/dz54q6j13k102m4gr5nfnv1r0000gn/T/ipykernel_48537/2487480544.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['navn'] = filtered_df['FIRSTNAME'].str.cat(filtered_df['SURNAME'], sep=' ')
